In [ ]:
#imorting modules
import keras
from keras.layers import Dense,Activation,Dropout,Conv2D,Flatten,MaxPooling2D,TimeDistributed,LSTM
from keras.models import Sequential
import librosa
import librosa.display
import numpy as np 
import os
import pandas as pd 
import tensorflow as tf
from glob import glob
import IPython
import matplotlib.pyplot as plt
from IPython.display import Audio


In [ ]:
#loading into memory 
zero=glob("../input/tensorflow-speech-recognition-challenge/train/audio/zero" + '/*.wav')
one=glob("../input/tensorflow-speech-recognition-challenge/train/audio/one" + '/*.wav')
two=glob("../input/tensorflow-speech-recognition-challenge/train/audio/two" + '/*.wav')
three=glob("../input/tensorflow-speech-recognition-challenge/train/audio/three" + '/*.wav')
four=glob("../input/tensorflow-speech-recognition-challenge/train/audio/four" + '/*.wav')
five=glob("../input/tensorflow-speech-recognition-challenge/train/audio/five" + '/*.wav')
six=glob("../input/tensorflow-speech-recognition-challenge/train/audio/six" + '/*.wav')
seven=glob("../input/tensorflow-speech-recognition-challenge/train/audio/seven" + '/*.wav')
eight=glob("../input/tensorflow-speech-recognition-challenge/train/audio/eight" + '/*.wav')
nine=glob("../input/tensorflow-speech-recognition-challenge/train/audio/nine" + '/*.wav')


In [ ]:
#labelling`
one1=[]

for i in range(0,len(zero)):
    one1.append((zero[i],0))
for i in range(0,len(one)):
    one1.append((one[i],1))
for i in range(0,len(two)):
    one1.append((two[i],2))
for i in range(0,len(three)):
    one1.append((three[i],3))
for i in range(0,len(four)):
    one1.append((four[i],4))
for i in range(0,len(five)):
    one1.append((five[i],5))
for i in range(0,len(six)):
    one1.append((six[i],6))
for i in range(0,len(seven)):
    one1.append((seven[i],7))
for i in range(0,len(eight)):
    one1.append((eight[i],8))
for i in range(0,len(nine)):
    one1.append((nine[i],9))


In [ ]:
#PREPROCESSING
D1=[]
dataframe=pd.DataFrame(one1,columns=["a","b"])
for row in range(0,len(dataframe)):
        y,sr = librosa.load(dataframe["a"][row],duration=1)
        ps = librosa.feature.melspectrogram(y=y,sr=sr)
        ps=librosa.util.fix_length(ps,44)
        D1.append((ps,dataframe["b"][row]))
len(D1)        

In [ ]:
#MODELING THE DATA
df2=pd.DataFrame(D1,columns=["ab","bc"])
df2=df2.sample(frac=1)
train=df2[:19000]
test=df2[19000:23000]
valid=df2[23000:23666]
train_x=train["ab"].values
test_x=test["ab"].values
valid_x=valid["ab"].values
train_x = np.array([x.reshape( (128, 44, 1) ) for x in train_x])
train_y=train["bc"].values
test_x = np.array([x.reshape( (128, 44, 1) ) for x in test_x])
test_y=test["bc"].values
valid_x = np.array([x.reshape( (128, 44, 1) ) for x in valid_x])
valid_y=valid["bc"].values

In [ ]:
#MODEL
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),input_shape=[128,44,1],activation="relu"),
    tf.keras.layers.MaxPooling2D(1,1),
     tf.keras.layers.Conv2D(32,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(10,activation="softmax"),
])

In [ ]:
#compiling and fitting
model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy'])

history=model.fit(
    x=train_x, 
    y=train_y,
    epochs=16,
    batch_size=128,validation_data=(valid_x,valid_y))
score = model.evaluate(
    x=test_x,
    y=test_y)

print('Test loss:', score[0])
print('Test accuray:', score[1])
model.save('D:\\my_modelsounds.h5')

In [ ]:
#intitution for number 4
sample="../input/foursound123/four.wav"
y,sr = librosa.load(sample,duration=1)
ps = librosa.feature.melspectrogram(y=y,sr=sr)
ps=librosa.util.fix_length(ps,44)
ps=np.reshape(ps,(1,128,44,1))
Audio(y,filename=sample)



In [ ]:
#waveplot
librosa.display.waveplot(y, sr=sr)

In [ ]:
#filter
S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)
librosa.display.specshow(librosa.power_to_db(S,ref=np.max),y_axis='mel', fmax=8000,
                         x_axis='time')

In [ ]:
#probabilities
l=model.predict(ps)
print(l)

In [ ]:
#class label
predict = np.array(l)
predict_class = np.argmax(predict, axis=1)
predict_class = predict_class.tolist()
predict_class